In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./2.url.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        for b in range(666):
            try:
                import requests

                import sys
                sys.path.append('../00.Tools')
                from crawler_configuration import get_header, get_proxy

                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(10, 10)).text

                # = = = = = = = = = = = = = = =

                part_code = resp.split('window.productId')[1].split(';')[0].split('=')[1].strip()

                # = = = = = = = = = = = = = = =

                import json

                json_ = json.loads(resp.split('<!-- Google structured data  Markup -->')[1].split('</script>')[0].split('<script type="application/ld+json">')[1])

                # = = = = = = = = = = = = = = =

                dict_src = {}
                list_src = [f'https:{image.strip()}' for image in json_['image']]
                for i, src in zip(range(len(list_src)), list_src):
                    dict_src[str(i)] = src

                # = = = = = = = = = = = = = = =

                sku = json_['sku'].strip()

                # = = = = = = = = = = = = = = =

                part_number = json_['mpn'].strip()

                # = = = = = = = = = = = = = = =

                brand = json_['brand']['name'].strip()

                # = = = = = = = = = = = = = = =

                price = f'''{json_['offers']['priceCurrency'].strip()} {json_['offers']['price']}'''

                # = = = = = = = = = = = = = = =

                from bs4 import BeautifulSoup
                from lxml import etree

                soup = BeautifulSoup(resp.strip(), 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                tag = html.xpath('//div[@id="productViewedLogHelper"]/div[@class="row"]/div[@class="col-12"]/div[@class="row"]/div[@class="col-8"]/p[@class="product-title"]/text()')[0].strip()

                # = = = = = = = = = = = = = = =

                title = html.xpath('//div[@id="productViewedLogHelper"]/div[@class="row"]/div[@class="col-12"]/div/h1/text()')[0].strip()

                # = = = = = = = = = = = = = = =

                detail = ''
                list_p = html.xpath('//div[@id="product-description-tab"]/div/p')
                list_ul = html.xpath('//div[@id="product-description-tab"]/div/ul')
                for p, ul in zip(list_p, list_ul):
                    if p.xpath('./strong/text()') and p.xpath('./strong/text()')[0].strip() == 'Part Details':
                        list_span = [text.strip() for text in ul.xpath('./li/span/text()')]
                        detail = '\n'.join(list_span[1:] if list_span[0].endswith(':') else list_span)

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Part Number': part_number,
                                         'Tag': tag,
                                         'Title': title,
                                         'SKU': sku,
                                         'Part Code': part_code,
                                         'Brand': brand,
                                         'Price': price,
                                         'Vehicle': '',
                                         'OEM': '',
                                         'Detail': detail,
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': json.dumps(dict_src)}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'

                # = = = = = = = = = = = = = = =

                break
            except:
                continue

        # = = = = = = = = = = = = = = =

        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b+1:3d}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if len(output_correct) != 0:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./3.part-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if len(output_error) != 0:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：2

[状态：ok，尝试次数：  1] - 593.https://www.1aauto.com/ford-front-driver-side-door-weatherstrip-seal-fairchild-automotive-f3004/i/1awsd00176
[剩余数量：0] - [当前时间：17:14:58]

[状态：ok，尝试次数：  1] - 1063.https://www.1aauto.com/1978-81-chevrolet-pontiac-roofrail-weatherstrip-seal-2-piece-set-metro-moulded-parts-rr-5020/i/1awsr00095
[剩余数量：0] - [当前时间：17:14:59]

输出ing...

Done ~
